In [ ]:
import os
import time
import glob
import subprocess

In [ ]:
path_to_videos = "/home/SharedFolder/CurrentDatasets/bdd100k_video_samples"
suffix_video_left = "_train_A_ganaug_050_over_augonlyasbase"
suffix_video_right = "_train_A_ganaug_050_over"
suffix_video_combined = suffix_video_left + suffix_video_right

In [ ]:
left_videos = glob.glob(path_to_videos + "/*" + suffix_video_left + ".mp4")
right_videos = glob.glob(path_to_videos + "/*" + suffix_video_right + ".mp4")
left_videos_no_suffix = [video.split(suffix_video_left)[0] for video in left_videos]
right_videos_no_suffix = [video.split(suffix_video_right)[0] for video in right_videos]
relevant_videos_no_suffix = sorted(list(set(left_videos_no_suffix) & set(right_videos_no_suffix)))
print(f"Found {len(relevant_videos_no_suffix)} videos for stacking.")

In [ ]:
def process_video(relevant_video_no_suffix, suffix_video_left, suffix_video_right, suffix_video_combined):
    left_video = relevant_video_no_suffix + suffix_video_left + ".mp4"
    right_video = relevant_video_no_suffix + suffix_video_right + ".mp4"
    path_name = os.path.dirname(left_video)
    target_path = os.path.join(path_name, "combined")
    combined_video = os.path.join(target_path, relevant_video_no_suffix.split(os.sep)[-1] + suffix_video_combined + ".mp4")
    if not os.path.isdir(target_path):
        os.makedirs(target_path, exist_ok = True)
    elif os.path.exists(combined_video):
        # do nothing if file already processed
        return -1
    # combine videos
    bash_cmd = ["ffmpeg", "-i", left_video, "-i", right_video, "-filter_complex", "hstack=inputs=2", "-c:v", "libx264", "-crf", "18", combined_video]
    subprocess.call(bash_cmd)
    return 0

In [ ]:
for i in range(len(relevant_videos_no_suffix)):
    print(f"Processing video {i + 1} of {len(relevant_videos_no_suffix)}", end = "")
    tic = time.time()
    result = process_video(relevant_videos_no_suffix[i], suffix_video_left, suffix_video_right, suffix_video_combined)
    toc = time.time()
    if result == 0:
        print(f"... done in {toc - tic:.2f}s")
    else:
        print(f"... skipped. File exists.")